In [3]:
# imports
import pandas as pd
import numpy as np
import librosa
import librosa.display
import os
import jams


pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='ticks')

import IPython.display as ipd
from IPython.display import Audio

# Data extraction model 2:
Get 500ms chroma vectors for all the sngs and corresponding labels
Keep all songs for this model (not only to_keep = True)

Steps:
- Define paths
- Load dataframe `chords_comp` 'containing all comp (accompagnement) data
- For each audio file where name appears in the dataframe)
-- slice on a 200ms time spam
-- create chromagram image for each frame 
-- get the corresponding label


### Define Paths, load DF

In [93]:
path_audio = "audio_hex-pickup_debleeded/"
audio_files = sorted(os.listdir(path_audio))
path_annotations = "annotation/"
path_images = "images_2/"
path_labels = "labels_2/"

In [38]:
# load df
df = pd.read_csv("labels/chords_comp.csv", index_col=0)
df = df.dropna()

In [41]:
df.head()

,id,track_name,time,duration,chord_instructed,chord_played,to_keep
0,00_BN1-129-Eb_comp_0,00_BN1-129-Eb_comp,0.000000,7.441860,D#:maj,D#:sus2(7)/1,False
1,00_BN1-129-Eb_comp_1,00_BN1-129-Eb_comp,7.441860,3.720930,G#:maj,G#:maj6(*5)/1,True
2,00_BN1-129-Eb_comp_2,00_BN1-129-Eb_comp,11.162791,3.720930,D#:maj,D#:maj7/1,True
3,00_BN1-129-Eb_comp_3,00_BN1-129-Eb_comp,14.883721,1.860465,A#:maj,A#:maj/1,True
4,00_BN1-129-Eb_comp_4,00_BN1-129-Eb_comp,16.744186,1.860465,G#:maj,"G#:maj6(2,b5,*5)/1",True


### Loop through audio files, get chromagrams and labels
- define params: length we want (200 ms)
- for each song in the list of songs we're using (len(audio_files)
- create a dataframe with track names as index
-- load in librosa
-- get length of the song (len(data)/sr
-- compute list of starts
-- create image for each start
-- get chord sequence and index it to the dataframe

In [99]:
# get audio_file_comp only
audio_file_comp = []
for file in audio_files:
    if 'comp' in str(file):
        audio_file_comp.append(file)

In [139]:
# get dataframe to store chord sequences

chord_seq_df = pd.DataFrame({'track':audio_file_comp, 'chord_sequ':''})
chord_seq_df = chord_seq_df.set_index('track')
chord_seq_df[:5]

,chord_sequ
track,
00_BN1-129-Eb_comp_hex_cln.wav,
00_BN1-147-Gb_comp_hex_cln.wav,
00_BN2-131-B_comp_hex_cln.wav,
00_BN2-166-Ab_comp_hex_cln.wav,
00_BN3-119-G_comp_hex_cln.wav,


In [127]:
# define params (take 0.5 sec for now - to try code)
dur = 0.5

In [142]:
# loop through data to extract pictures and labels

for file in audio_file_comp:
    print(f'file {file}')        
    track_path = os.path.join(path_audio, file)
    
    # load in librosa
    data, sr = librosa.load(track_path)
    length = len(data)/sr 
    
    # compute list of starts for the audio file
    start_list = [0]
    k = 0
    while k+dur<length:
        k += dur
        start_list.append(k)
    print(f'total images: {len(start_list)}')  
    
    # get list of labels associated
    chord_sequence = []
    for start in start_list:
        # iterate through starts for the song
        data_track = df[df['track_name'] == file[:-12]]
        for index, row in data_track.iterrows():
            if (start >= float(row['time'])) and (start < (float(row['time']) + float(row['duration']))):
                chord_sequence.append(row['chord_instructed'])
    
    print(f'total labels: {len(chord_sequence)}')
    
    # create an image for each start of the audio file    
    for j in range(len(start_list)):
        data, sr = librosa.load(path_audio + file, offset=start_list[j] , duration=dur)
        chromagram = librosa.feature.chroma_cens(data, sr=sr)
        chromagram_mag = librosa.magphase(chromagram)[0]**4
        
        # save image
        name = file[:-12] + '_' + str(j)
        fig = plt.figure(figsize=(2, 2))
        librosa.display.specshow(chromagram_mag, sr=sr, cmap='gray')
        fig.savefig(path_images + name)
        plt.close()
        
    chord_seq_df.loc[file, 'chord_sequ'] = chord_sequence

file 00_BN1-129-Eb_comp_hex_cln.wav
total images: 45
total labels: 45


/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=690
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=345
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=173
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=895
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=448
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=224
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWar

file 00_BN1-147-Gb_comp_hex_cln.wav
total images: 40
total labels: 40


/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1887
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=944
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=472
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=236
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=118
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=59
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWar

file 00_BN2-131-B_comp_hex_cln.wav
total images: 59
total labels: 59


...

file 00_SS2-88-F_comp_hex_cln.wav
total images: 88
total labels: 88
file 00_SS3-84-Bb_comp_hex_cln.wav
total images: 92


/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=573
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=287
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=144
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=72
  warnings.warn(


total labels: 92
file 00_SS3-98-C_comp_hex_cln.wav
total images: 79
total labels: 79
file 01_BN1-129-Eb_comp_hex_cln.wav
total images: 45
total labels: 45
file 01_BN1-147-Gb_comp_hex_cln.wav
total images: 40
total labels: 40
file 01_BN2-131-B_comp_hex_cln.wav
total images: 59
total labels: 59
file 01_BN2-166-Ab_comp_hex_cln.wav
total images: 47
total labels: 47
file 01_BN3-119-G_comp_hex_cln.wav
total images: 65
total labels: 65
file 01_BN3-154-E_comp_hex_cln.wav
total images: 50
total labels: 50
file 01_Funk1-114-Ab_comp_hex_cln.wav
total images: 51
total labels: 51
file 01_Funk1-97-C_comp_hex_cln.wav
total images: 60
total labels: 60
file 01_Funk2-108-Eb_comp_hex_cln.wav
total images: 72
total labels: 73
file 01_Funk2-119-G_comp_hex_cln.wav
total images: 65
total labels: 65
file 01_Funk3-112-C#_comp_hex_cln.wav
total images: 69
total labels: 69
file 01_Funk3-98-A_comp_hex_cln.wav
total images: 79
total labels: 79
file 01_Jazz1-130-D_comp_hex_cln.wav
total images: 45
total labels: 45


/opt/conda/lib/python3.8/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


file 05_Funk1-97-C_comp_hex_cln.wav
total images: 60
total labels: 60
file 05_Funk2-108-Eb_comp_hex_cln.wav
total images: 72
total labels: 73
file 05_Funk2-119-G_comp_hex_cln.wav
total images: 65
total labels: 65
file 05_Funk3-112-C#_comp_hex_cln.wav
total images: 69
total labels: 69
file 05_Funk3-98-A_comp_hex_cln.wav
total images: 79
total labels: 79
file 05_Jazz1-130-D_comp_hex_cln.wav
total images: 45
total labels: 45
file 05_Jazz1-200-B_comp_hex_cln.wav
total images: 29
total labels: 30
file 05_Jazz2-110-Bb_comp_hex_cln.wav
total images: 70
total labels: 70
file 05_Jazz2-187-F#_comp_hex_cln.wav
total images: 42
total labels: 42
file 05_Jazz3-137-Eb_comp_hex_cln.wav
total images: 57
total labels: 57
file 05_Jazz3-150-C_comp_hex_cln.wav
total images: 52
total labels: 52
file 05_Rock1-130-A_comp_hex_cln.wav
total images: 45
total labels: 45
file 05_Rock1-90-C#_comp_hex_cln.wav
total images: 64
total labels: 64
file 05_Rock2-142-D_comp_hex_cln.wav
total images: 55
total labels: 55
fil

In [143]:
chord_seq_df

,chord_sequ
track,
00_BN1-129-Eb_comp_hex_cln.wav,"[D#:maj, D#:maj, D#:maj, D#:maj, D#:maj, D#:ma..."
00_BN1-147-Gb_comp_hex_cln.wav,"[F#:maj, F#:maj, F#:maj, F#:maj, F#:maj, F#:ma..."
00_BN2-131-B_comp_hex_cln.wav,"[E:min, E:min, E:min, E:min, A:7, A:7, A:7, A:..."
00_BN2-166-Ab_comp_hex_cln.wav,"[C#:min, C#:min, C#:min, F#:7, F#:7, F#:7, B:m..."
00_BN3-119-G_comp_hex_cln.wav,"[G:maj, G:maj, G:maj, G:maj, G:maj, D:maj, D:m..."
...,...
05_SS1-68-E_comp_hex_cln.wav,"[E:maj, E:maj, E:maj, E:maj, E:maj, E:maj, E:m..."
05_SS2-107-Ab_comp_hex_cln.wav,"[C#:min, C#:min, C#:min, C#:min, C#:min, F#:7,..."
05_SS2-88-F_comp_hex_cln.wav,"[A#:min, A#:min, A#:min, A#:min, A#:min, A#:mi..."


In [148]:
# add length of sequence
chord_seq_df['len_sequ'] = chord_seq_df['chord_sequ'].apply(lambda x: len(x))

In [164]:
chord_seq_df.head()

,chord_sequ,len_sequ
track,,
00_BN1-129-Eb_comp_hex_cln.wav,"D#:maj, D#:maj, D#:maj, D#:maj, D#:maj, D#:maj...",45
00_BN1-147-Gb_comp_hex_cln.wav,"F#:maj, F#:maj, F#:maj, F#:maj, F#:maj, F#:maj...",40
00_BN2-131-B_comp_hex_cln.wav,"A:7, A:7, D:maj, D:maj, D:maj, G:maj, G:maj, G...",59
00_BN2-166-Ab_comp_hex_cln.wav,"B:maj, B:maj, B:maj, E:maj, E:maj, E:maj, A#:h...",47
00_BN3-119-G_comp_hex_cln.wav,"D:maj, D:maj, D:maj, E:min, E:min, E:min, E:mi...",65


In [158]:
chord_seq_df.dtypes

chord_sequ    object
len_sequ       int64
dtype: object

In [162]:
# remove, [, ], ' from the chord_seq as it's stored as a string...
chord_seq_df['chord_sequ'] = chord_seq_df['chord_sequ'].apply(lambda x: str(x).replace('[',''))
chord_seq_df['chord_sequ'] = chord_seq_df['chord_sequ'].apply(lambda x: str(x).replace(']',''))
chord_seq_df['chord_sequ'] = chord_seq_df['chord_sequ'].apply(lambda x: str(x).replace("'",''))

In [163]:
chord_seq_df.head()

,chord_sequ,len_sequ
track,,
00_BN1-129-Eb_comp_hex_cln.wav,"D#:maj, D#:maj, D#:maj, D#:maj, D#:maj, D#:maj...",45
00_BN1-147-Gb_comp_hex_cln.wav,"F#:maj, F#:maj, F#:maj, F#:maj, F#:maj, F#:maj...",40
00_BN2-131-B_comp_hex_cln.wav,"A:7, A:7, D:maj, D:maj, D:maj, G:maj, G:maj, G...",59
00_BN2-166-Ab_comp_hex_cln.wav,"B:maj, B:maj, B:maj, E:maj, E:maj, E:maj, A#:h...",47
00_BN3-119-G_comp_hex_cln.wav,"D:maj, D:maj, D:maj, E:min, E:min, E:min, E:mi...",65


In [150]:
# get the min and max length
print(min(chord_seq_df['len_sequ']))
print(max(chord_seq_df['len_sequ']))

30
92


In [ ]:
# we'll be using only a length of 30 for the model

In [165]:
# export dataframe
chord_seq_df.to_csv(path_labels + "chords_seq.csv")